In [ ]:
import streamlit as st
import pandas as pd
import numpy as np

import joblib
import shap
import matplotlib.pyplot as plt
from textblob import TextBlob
from sklearn.preprocessing import StandardScaler
from transformers import pipeline

In [ ]:
st.title("🍽️ Restaurant Risk Prediction Dashboard")


st.sidebar.header("Select Restaurant")
df = pd.read_csv("cleaned_reviews.csv")
restaurant_ids = df['restaurant_id'].unique()
selected_id = st.sidebar.selectbox("Restaurant ID:", restaurant_ids)


2025-09-02 07:25:02.363 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:25:02.364 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:25:02.365 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:25:02.366 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:25:02.367 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:25:02.368 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:25:02.438 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:25:02.442 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
#drift det setup, build vocab
import re

def build_vocab(texts):
    vocab = set()
    for txt in texts:
        words = re.findall(r'\b\w+\b', txt.lower())
        vocab.update(words)
    return vocab


training_vocab = build_vocab(df['review_text'].tolist())


def detect_drift(reviews, vocab, threshold=0.15):
    reviews_words = []
    for txt in reviews:
        words = re.findall(r'\b\w+\b', txt.lower())
        reviews_words.extend(words)

    if not reviews_words:
        return False, 0.0

    unknown_words = [w for w in reviews_words if w not in vocab]
    fraction_unknown = len(unknown_words) / len(reviews_words)
    drift_flag = fraction_unknown > threshold
    return drift_flag, fraction_unknown



In [ ]:
#feature eng
df['sentiment'] = df['review_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['review_length'] = df['review_text'].apply(lambda x: len(x.split()))

feature_columns = ['monthly_sales', 'complaints_count', 'avg_rating', 'staff_turnover', 'review_length', 'sentiment']
X = df[feature_columns]
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=feature_columns)

In [ ]:
model = joblib.load("restaurant_risk_xgb.pkl")
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_scaled)

In [ ]:
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn"
)

Device set to use cpu


In [ ]:
restaurant_data = df[df['restaurant_id'] == selected_id]
X_restaurant = X_scaled.loc[restaurant_data.index]

In [ ]:
if st.sidebar.button("Detect Drift"):
    drift_flag, fraction_unknown = detect_drift(
        restaurant_data['review_text'].tolist(),
        training_vocab
    )
    if drift_flag:
        st.warning(f"⚠️ Drift detected! {fraction_unknown*100:.2f}% of words are unusual compared to training data.")
    else:
        st.success(f"✅ No significant drift detected. ({fraction_unknown*100:.2f}% unusual words)")


2025-09-02 07:27:37.987 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:27:37.989 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:27:37.990 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:27:37.991 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:27:37.992 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:27:37.992 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
if st.button("Generate Summary"):
    reviews_text = " ".join(restaurant_data['review_text'].tolist())
    if len(reviews_text.split()) > 50:
        summary = summarizer(reviews_text, max_length=100, min_length=30, do_sample=False)[0]['summary_text']
    else:
        summary = reviews_text
    st.subheader("Reviews Summary")
    st.write(summary)

    st.subheader("Feature Values for Selected Restaurant")
    features_to_show = X_restaurant.iloc[0]
    for feat in feature_columns:
        st.write(f"{feat}: {features_to_show[feat]:.3f}")

2025-09-02 07:27:58.810 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:27:58.817 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:27:58.819 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:27:58.822 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:27:58.824 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:27:58.827 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
if st.button("Predict Risk"):
    prediction = model.predict(X_restaurant)[0]
    risk = "High Risk ⚠️" if prediction == 1 else "Low Risk ✅"
    st.subheader("Risk Prediction")
    st.write(risk)

    shap_vals_restaurant = shap_values[restaurant_data.index, :]
    mean_shap = np.abs(shap_vals_restaurant).mean(axis=0)
    top_indices = mean_shap.argsort()[-3:][::-1]
    feature_array = np.array(feature_columns)
    top_features = feature_array[top_indices]

    st.subheader("Top 3 Explanatory Features")
    for i, feat in enumerate(top_features, 1):
        st.write(f"{i}. {feat}")

    st.subheader("SHAP Summary Plot")
    fig, ax = plt.subplots()
    shap.summary_plot(shap_vals_restaurant, X_restaurant, show=False)
    st.pyplot(fig)


2025-09-02 07:28:20.182 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:28:20.184 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:28:20.185 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:28:20.185 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:28:20.186 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 07:28:20.187 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
